In [1]:
pip install together


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
MODEL_NAME='meta-llama/Llama-2-7b-chat-hf'

In [4]:
import getpass

In [5]:
TOGETHER_API_KEY = 'fa12a40c9ee86b002d11b0f348c884bb04780563335f17e7491bf3b006dc302d'

In [6]:
import os
from together import Together

In [7]:
experiment_A_prompt = """The following is an excerpt from Fleming’s first James Bond novel, "Casino Royale". In this book, Bond has been assigned to ‘ruin’ a criminal figure named Le Chiffre by, as it happens, causing Le Chiffre to lose a considerable amount of money gambling. Along the way, Bond has acquired a lady interest named Vesper. Although Bond has, in fact, brought about the gambling losses, Le Chiffre has laid a successful trap for Bond. Bond and Vesper are now the prisoners of Le Chiffre and his two gunmen.

\"\"\"
{STORY}
\"\"\"

Use the passage above to answer the following questions:

***Question 1: "How likely is it that Bond will escape from Le Chiffre?"***
1. Not very likely
2. Somewhat likely
3. Moderately likely
4. Likely
5. Quite likely
6. Very likely
7. Extremely likely

***Question 2: "How suspenseful do you find this passage to be?"***
1. Not very suspenseful
2. Somewhat suspenseful
3. Moderately suspenseful
4. Suspenseful
5. Quite suspenseful
6. Very suspenseful
7. Extremely suspenseful

Answer the questions using the following RESPONSE FORMAT, then stop generating:
RESPONSE FORMAT: \"\"\"
Q1: [1-7]
Q2: [1-7]
\"\"\"
"""

In [19]:
prompts = {
    "Experiment A": experiment_A_prompt
}

In [8]:
experiment_A_pen_not_mentioned = """Le Chiffre was standing in the doorway of a room on the right. He crooked a finger at Bond in a silent, spidery summons.
  Vesper was being led down a passage towards the back of the house. Bond suddenly decided. With a backward kick which connected with the thin man’s shins and brought a whistle of pain from him, Bond hurled himself down the passage after her. His plan was to do as much damage as possible to the two gunmen and be able to exchange a few hurried words with the girl.
  Like lightning the Corsican slammed himself back against the wall of the passage and, as Bond’s foot whistled past his hip, he very quickly, but somehow delicately, shot out his left hand, caught Bond’s shoe at the top of its arc, and twisted it sharply. As he crashed to the ground, Bond rolled agilely and, with a motion in which he took great pride, he righted himself with minimal damage.
  “Search him.” barked Le Chiffre.
  The two gunmen dragged Bond to his feet. While the thin man kept his gun trained on Bond’s unquiet chest, the Corsican roughly stripped Bond’s revolver out of its shoulder holster. He twisted Bond around brusquely in search of other weaponry. Le Chiffre observed his assistant’s work attentively. Then, as if reading Bond’s thoughts, he crossed the room and said, “Come my dear friend. Let’s not waste time.”"""

experiment_A_pen_mentioned_removed = """  Le Chiffre was standing in the doorway of a room on the right. He crooked a finger at Bond in a silent, spidery summons.
  Vesper was being led down a passage towards the back of the house. Bond suddenly decided. With a backward kick which connected with the thin man’s shins and brought a whistle of pain from him, Bond hurled himself down the passage after her. His plan was to do as much damage as possible to the two gunmen and be able to exchange a few hurried words with the girl.
  Like lightning the Corsican slammed himself back against the wall of the passage and, as Bond’s foot whistled past his hip, he very quickly, but somehow delicately, shot out his left hand, caught Bond’s shoe at the top of its arc, and twisted it sharply. As he crashed to the ground, Bond rolled agilely and, with a motion that he hoped went unnoticed, moved his fountain pen deeper into his breast pocket.
  “Search him.” barked Le Chiffre.
  The two gunmen dragged Bond to his feet. While the thin man kept his gun trained on Bond’s unquiet chest, the Corsican roughly stripped Bond’s revolver out of its shoulder holster. He twisted Bond around brusquely in search of other weaponry. Le Chiffre observed his assistant’s work attentively. Then, as if reading Bond’s thoughts, he crossed the room and snatched away Bond’s fountain pen. “Come my dear friend,” said Le Chiffre. “Let’s not waste time.”"""



In [20]:
version_prompts = {
    "Experiment A": [experiment_A_pen_not_mentioned,experiment_A_pen_mentioned_removed]
}

In [9]:
print(experiment_A_prompt.format(STORY=experiment_A_pen_not_mentioned))

The following is an excerpt from Fleming’s first James Bond novel, "Casino Royale". In this book, Bond has been assigned to ‘ruin’ a criminal figure named Le Chiffre by, as it happens, causing Le Chiffre to lose a considerable amount of money gambling. Along the way, Bond has acquired a lady interest named Vesper. Although Bond has, in fact, brought about the gambling losses, Le Chiffre has laid a successful trap for Bond. Bond and Vesper are now the prisoners of Le Chiffre and his two gunmen.

"""
Le Chiffre was standing in the doorway of a room on the right. He crooked a finger at Bond in a silent, spidery summons.
  Vesper was being led down a passage towards the back of the house. Bond suddenly decided. With a backward kick which connected with the thin man’s shins and brought a whistle of pain from him, Bond hurled himself down the passage after her. His plan was to do as much damage as possible to the two gunmen and be able to exchange a few hurried words with the girl.
  Like li

In [10]:
client = Together(api_key=TOGETHER_API_KEY)

In [21]:
def get_response(experiment, version):
    response = client.chat.completions.create(
        model="meta-llama/Llama-2-7b-chat-hf",
        messages=[
            {"role": "user", "content": experiment.format(STORY=version)}
        ]
    )
    return response

In [26]:

n = 10
responses = []
for _ in range(n):
    for key, el in prompts.items():
        if version_prompts[key]:
            for ver_el in version_prompts[key]:
                response = get_response(prompts[key], ver_el)
                responses.append(response)

In [28]:
import re
import json
response_data = []
for idx, resp in enumerate(responses):
    content = resp.choices[0].message.content
    numbers = re.findall(r'\[(\d+)\]', content)
    response_data.append({
        "response_number": idx + 1,
        "Q1": numbers[0] if len(numbers) > 0 else None,
        "Q2": numbers[1] if len(numbers) > 1 else None
    })

# Save to JSON file
with open('responses.json', 'w') as json_file:
    json.dump(response_data, json_file, indent=4)